In [ ]:


import pandas as pd
from wikiwho_chobj import Chobjer

#import nltk
#nltk.download("stopwords")

co = Chobjer(article="39570", pickles_path='', lang='en', context=5)
df = pd.DataFrame(co.iter_chobjs(), columns = next(co.iter_chobjs()).keys())

#jlogie = pd.read_csv("../John_Logie_Baird_FULL.csv")

In [ ]:
jlogie = pd.read_csv("../../John_Logie_Baird_FULL.csv")

In [ ]:
import numpy as np

def combine(chobj):
    # to be called by an apply function on a dataframe of change objects as provided by wikiwho
    # depends on jlogie as ground truth labels
    boolean = jlogie["rev_id"] == chobj["to_rev"]
    token = jlogie[boolean]    
    if not token.empty and len(token) == 1:
        which_jlogie = token["token_id"].isin(chobj["ins_tokens"])
        if np.sum(which_jlogie) == 1:
            to_merge = jlogie.iloc[which_jlogie.index[0]]
            chobj["nationality"] = to_merge["nationality"]
            chobj["birth_place"] = to_merge["birth_place"]
            chobj["Link"] = to_merge["Link"]
            chobj["Bulk"] = to_merge["Bulk"]
            chobj["token"] = to_merge["token"]
            chobj["action"] = to_merge["action"]
            return chobj
        elif np.sum(which_jlogie) > 1:
            print("more than one row in jlogie found!")
            return pd.Series(None)
        elif np.sum(which_jlogie) == 0:
            chobj["nationality"] = None
            chobj["birth_place"] = None
            chobj["Link"] = None
            chobj["Bulk"] = None
            chobj["token"] = None
            chobj["action"] = None
            return chobj
    elif not token.empty and len(token) > 1:
        which_jlogie = token["token_id"].isin(chobj["ins_tokens"])
        if np.sum(which_jlogie) == 1:
            to_merge = jlogie.iloc[which_jlogie.index[0]]
            chobj["nationality"] = to_merge["nationality"]
            chobj["birth_place"] = to_merge["birth_place"]
            chobj["Link"] = to_merge["Link"]
            chobj["Bulk"] = to_merge["Bulk"]
            chobj["token"] = to_merge["token"]
            chobj["action"] = to_merge["action"]
            return chobj
        elif np.sum(which_jlogie) == 0:
            chobj["nationality"] = None
            chobj["birth_place"] = None
            chobj["Link"] = None
            chobj["Bulk"] = None
            chobj["token"] = None
            chobj["action"] = None
            return chobj
        elif np.sum(which_jlogie) > 1:
            for col in ["nationality", "birth_place", "Link", "Bulk"]:
                if len(token[col].unique()) == 1:
                    chobj[col] = list(token[col])[0]
                else:
                    chobj[col] = None
                    print("non congruent values found for df['to_rev'] == ", str(chobj["to_rev"]), " and token ids: ", list(token["token_id"]), " in jlogie. Setting None to column ", str(col))
            return chobj
        return pd.Series(None)
    else:
        chobj["nationality"] = None
        chobj["birth_place"] = None
        chobj["Link"] = None
        chobj["Bulk"] = None
        chobj["token"] = None
        chobj["action"] = None
        return chobj

merged = df.apply(lambda x: combine(x), axis=1)
merged = merged.dropna(how="all")

In [ ]:
len(merged)

In [ ]:
len(df)

##  Embed words by creating a vector of length 300 for each inserted and deleted tokens, so the resulting vector for one change object has length 600

In [ ]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

embed = load_vectors('../../wiki-news-300d-1M-subword.vec')

In [ ]:
from nltk.corpus import stopwords
import numpy as np
from gensim.sklearn_api import W2VTransformer
from gensim.models import KeyedVectors
from copy import deepcopy
import pdb

WORD_EMBED_SIZE = 300
LEFT_CONTEXT = 5
RIGHT_CONTEXT = 5
GAP = True

def transform(phrase : list, embedding):
    li_vecs = []
    for i in range(len(phrase)):
        if phrase[i] in embedding:
            li_vecs.append(list(deepcopy(embedding[phrase[i]])))
    if len(li_vecs) != 0:
        vecs = np.stack(li_vecs)
        return vecs            
    else:
        return None

def filter_stopwords(phrase):
    important_words = []
    for word in phrase:
        if word not in stopwords.words('english'):
            important_words.append(word)
    return important_words

def create_features(chobj, use_gap, left_context, right_context):
    if left_context > 0:
        left_wordvecs = transform(filter_stopwords(list(chobj["left_token_str"][-left_context:])), embed)
        if left_wordvecs is None:
            left_wordvecs = np.full(WORD_EMBED_SIZE, 0)
        else:
            left_wordvecs = np.mean(left_wordvecs, axis=0)
    if right_context > 0:
        right_wordvecs = transform(filter_stopwords(list(chobj["right_token_str"][:right_context])), embed)  
        if right_wordvecs is None:
            right_wordvecs = np.full(WORD_EMBED_SIZE, 0)
        else:
            right_wordvecs = np.mean(right_wordvecs, axis=0)
    if use_gap:
        ins_wordvecs = transform(filter_stopwords(list(chobj["ins_tokens_str"])), embed)     
        del_wordvecs = transform(filter_stopwords(list(chobj["del_tokens_str"])), embed)
        if ins_wordvecs is None:
            ins_wordvecs = np.full(WORD_EMBED_SIZE, 0)
        else:
            ins_wordvecs = np.mean(ins_wordvecs, axis=0)
        if del_wordvecs is None:
            del_wordvecs = np.full(WORD_EMBED_SIZE, 0)
        else:
            del_wordvecs = np.mean(del_wordvecs, axis=0)
    
    li = []
    for a in ["left_wordvecs", "right_wordvecs", "ins_wordvecs", "del_wordvecs"]:
        if a in vars():
            li.append(vars()[a])
    
    try:
        feat = pd.Series(np.nan_to_num(np.concatenate(li)))
    except ValueError:
        pdb.set_trace()
    
    return feat

Embedded = df.apply(lambda x: create_features(x, use_gap=GAP, left_context=LEFT_CONTEXT, right_context=RIGHT_CONTEXT), 
                        axis=1)

## K-means clustering

In [ ]:
from sklearn.cluster import KMeans

clusterer = KMeans(max_iter=3000, random_state=42)
clusters = clusterer.fit_predict(Embedded)

In [ ]:
pd.Series(clusters).unique()

## Visualization and plotting

In [ ]:
from sklearn.manifold import TSNE

X = TSNE(random_state=42).fit_transform(Embedded)

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import random

def convert_editors_to_colors(editors):
    ed_col = {}
    col_list = []
    for ed in editors:
        if ed in ed_col:
            col_list.append(ed_col[ed])
        else:
            r = lambda: random.randint(0,255)
            new_col = '#%02X%02X%02X' % (r(),r(),r())
            ed_col[ed] = new_col
            col_list.append(new_col)
            
    return col_list

In [ ]:
fig = plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
plt.scatter(X[:,0], X[:,1], s=10)

### interactively

In [ ]:
%matplotlib inline
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

# enable javascript support
init_notebook_mode(connected=True)

r = lambda: random.randint(0,255)

traces = []
for c in pd.Series(clusters).unique():
    

    # Create a trace
    trace = go.Scatter(
        x=X[clusters==c,0],
        y=X[clusters==c,1],
        mode = 'markers',
        name = str(c),
        marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r())),
        text = Embedded.index,
        showlegend = True,

    )
    traces.append(trace)

data = traces

# Plot and embed in ipython notebook
iplot(data, filename='basic-scatter')

In [ ]:
%matplotlib inline
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

# enable javascript support
init_notebook_mode(connected=True)

r = lambda: random.randint(0,255)

traces = []
for c in ["Y", "N", None]:
    
    if c is None:
        trace = go.Scatter(
            x=X[merged["nationality"].isna(),0],
            y=X[merged["nationality"].isna(),1],
            mode = 'markers',
            name = str(c),
            marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r())),
            text = Embedded.index,
            showlegend = True,
        )
    else:
        # Create a trace
        trace = go.Scatter(
            x=X[merged["nationality"]==c,0],
            y=X[merged["nationality"]==c,1],
            mode = 'markers',
            name = str(c),
            marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r())),
            text = Embedded.index,
            showlegend = True,
        )
    traces.append(trace)

data = traces

# Plot and embed in ipython notebook
iplot(data, filename='basic-scatter')

In [ ]:
None is None

In [ ]:
merged["nationality"].unique

In [ ]:
np.sum(Embedded.loc[[386, 626, 774, 75, 331]] <= 0, axis=1)

In [ ]:
Embedded.shape

In [ ]:
np.sum(np.sum(Embedded.iloc[:,600:1200] == 0.5, axis=1) == 600)

In [ ]:
df.loc[1179]["ins_tokens_str"]

In [ ]:
len(df)